In [4]:
import pickle
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import os 
from scipy.constants import N_A, pi

# Read in the data
with open("data.pkl", "rb") as f:
    data = pickle.load(f)
    
    # Inject into global namespace
    globals().update(data)


In [ ]:
import numpy as np

'''
    UNITS:
        mass_density: g/cm3
        atom_density: N/b-cm
        area: cm2
        cross sections: b
        power: W
        all dimensions: m
        molar_mass: g/mol

    Values for composition are (ZAID, volume fraction)
'''

reactor_data = {
    "cells": {
        "fuel":{
            "mass_density": 19.0,
            "number_density" : None,
            "area": pi * ((6.35/2)-0.04)**2,
            "composition": {}
        },

        "coolant":{
            "mass_density": 11.35,
            "atom_density" : 3.299e-2, # PNNL
            "area": np.pi * ((6.35/2)-0.04)**2,
            "composition": {
                ("82000", 1)
            },
        },
        
        "coolant":{
            "mass_density": 11.35,
            "atom_density" : 3.299e-2, # PNNL
            "area": np.pi * ((6.35/2)-0.04)**2,
            "composition": {
                ("82000", 1)
            },

        },

        "cladding":{}

    },

    "core": {
        "rho_fuel": 19.0,
        "OD_fuel": 6.35,          # Fuel + cladding outer diameter
        "Th_clad": 0.04,          # Cladding thickness
        "delta_core": 20.0,       # Extrapolation correction for core dimensions
        "p_th": 1.2e9             # Thermal power (W)
    },
    
    "ratios": {
        "phi_fuel_ratio": np.array([1.60, 1.60, 1.60, 1.45, 1.45, 1.34, 1.34, 1.34]),
        "phi_clad_ratio": np.array([1.01, 1.01, 1.01, 1.02, 1.02, 1.03, 1.03, 1.03]),
        "phi_cool_ratio": np.array([0.90, 0.90, 0.91, 0.91, 0.91, 0.94, 0.94, 0.94])
    },

    "energy_structure": {
        "lethargy_width": np.array([1.5, 1.0, 1.0, 1.0, 1.0, 1.0, 3.0, 0.0]),  # in eV
        "lower_energy_bounds": np.array([2.2e6, 820e3, 300e3, 110e3, 40e3, 15e3, 750, 0])
    },

    "nuclides": {
        "82000": {
            "molar_mass":
            "xs": {
                "sigma_tr": 1e-24 * np.array([1.5, 2.2, 3.6, 3.5, 4.0, 3.9, 7.3, 3.2]),
                "sigma_ng": 1e-24 * np.array([0.0050, 0.0002, 0.0004, 0.0010, 0.0010, 0.0010, 0.0090, 0.0080]),
                "sigma_f": 1e-24 * np.array([0 for _ in range(8)]),
                "sigma_r": 1e-24 * np.array([0.623, 0.6908, 0.4458, 0.2900, 0.3500, 0.3000, 0.0400, 0.0000]),
                "nu_sigma_f": None  # to be computed below
            },
            "yield": {
                "nu_f": np.array([0 for _ in range(8)])
            },
            "scattering": {
                "sigma_ss": 1e-24 * np.array([
                    [0.0000, 0.5200, 0.0900, 0.0030, 0.0090, 0.0010, 0.0000, 0.0000],
                    [0.0000, 0.0000, 0.6900, 0.0000, 0.0004, 0.0004, 0.0000, 0.0000],
                    [0.0000, 0.0000, 0.0000, 0.4400, 0.0050, 0.0008, 0.0000, 0.0000],
                    [0.0000, 0.0000, 0.0000, 0.0000, 0.2900, 0.0000, 0.0000, 0.0000],
                    [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.3500, 0.0000, 0.0000],
                    [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.3000, 0.0000],
                    [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0400],
                    [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000]
                ])
            }
        },

        "26000": {
            "xs": {
                "sigma_tr": 1e-24 * np.array([2.2, 2.1, 2.4, 3.1, 4.5, 6.1, 6.9, 10.4]),
                "sigma_ng": 1e-24 * np.array([0.0200, 0.0030, 0.0050, 0.0060, 0.0080, 0.0120, 0.0320, 0.0200]),
                "sigma_f": 1e-24 * np.array([0 for _ in range(8)]),
                "sigma_r": 1e-24 * np.array([1.0108, 0.4600, 0.1200, 0.1400, 0.2800, 0.0700, 0.0400, 0.0]),
                "nu_sigma_f": None  # to be computed below
            },
            "yield": {
                "nu_f": np.array([0 for _ in range(8)])
            },
            "scattering": {
                "sigma_ss": 1e-24 * np.array([
                    [0.0000, 0.7500, 0.2000, 0.5000, 0.0100, 0.0008, 0.0000, 0.0000],
                    [0.0000, 0.0000, 0.3300, 0.1000, 0.0200, 0.0100, 0.0000, 0.0000],
                    [0.0000, 0.0000, 0.0000, 0.1200, 0.0000, 0.0000, 0.0000, 0.0000],
                    [0.0000, 0.0000, 0.0000, 0.0000, 0.1400, 0.0000, 0.0000, 0.0000],
                    [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.2800, 0.0000, 0.0000],
                    [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0700, 0.0000],
                    [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0400],
                    [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000]
                ])
            }
        },

        "92238": {
            "xs": {
                "sigma_tr": 1e-24 * np.array([4.3, 4.8, 6.3, 9.3, 11.0, 12.0, 13.1, 11.0]),
                "sigma_ng": 1e-24 * np.array([0.0100, 0.0900, 0.1100, 0.1500, 0.2600, 0.4700, 0.8400, 1.4700]),
                "sigma_f": 1e-24 * np.array([0.58, 0.20, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00]),
                "sigma_r": 1e-24 * np.array([2.293, 1.490, 0.375, 0.293, 0.200, 0.090, 0.0100, 0.0000]),
                "nu_sigma_f": None  # to be computed below
            },
            "yield": {
                "nu_f": np.array([2.91, 2.58, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
            },
            "scattering": {
                "sigma_ss": 1e-24 * np.array([
                    [0.0000, 1.2800, 0.7800, 0.2000, 0.0300, 0.0030, 0.0000, 0.0000],
                    [0.0000, 0.0000, 1.0500, 0.4200, 0.0100, 0.0100, 0.0000, 0.0000],
                    [0.0000, 0.0000, 0.0000, 0.3300, 0.0400, 0.0050, 0.0009, 0.0000],
                    [0.0000, 0.0000, 0.0000, 0.0000, 0.2900, 0.0030, 0.0005, 0.0000],
                    [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.1800, 0.0200, 0.0000],
                    [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0900, 0.0000],
                    [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0100],
                    [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000]
                ])
            }
        },

        "94239": {
            "xs": {
                "sigma_tr": 1e-24 * np.array([4.5, 5.1, 6.3, 8.6, 11.0, 13.0, 16.0, 31.8]),
                "sigma_ng": 1e-24 * np.array([0.0100, 0.0300, 0.1100, 0.2000, 0.3500, 0.5900, 1.9800, 8.5400]),
                "sigma_f": 1e-24 * np.array([1.85, 1.82, 1.60, 1.51, 1.60, 1.67, 2.78, 10.63]),
                "sigma_r": 1e-24 * np.array([1.4950, 0.8260, 0.3709, 0.1905, 0.1500, 0.0900, 0.0100, 0.0000]),
                "nu_sigma_f": None  # to be computed below
            },
            "yield": {
                "nu_f": np.array([3.40, 3.07, 2.95, 2.90, 2.88, 2.88, 2.87, 2.87])
            },
            "scattering": {
                "sigma_ss": 1e-24 * np.array([
                    [0.0000, 0.6600, 0.6000, 0.1900, 0.0400, 0.0050, 0.0000, 0.0000],
                    [0.0000, 0.0000, 0.6400, 0.1500, 0.0300, 0.0060, 0.0000, 0.0000],
                    [0.0000, 0.0000, 0.0000, 0.3100, 0.0500, 0.0100, 0.0009, 0.0000],
                    [0.0000, 0.0000, 0.0000, 0.0000, 0.1800, 0.0100, 0.0005, 0.0000],
                    [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.1300, 0.0200, 0.0000],
                    [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0900, 0.0000],
                    [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0100],
                    [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
                ])
            }
        }
    }
}

# Compute nu_sigma_f for each nuclide
for nuc in reactor_data["nuclides"].values():
    xs = nuc["xs"]
    nu = nuc["yield"]["nu_f"]
    xs["nu_sigma_f"] = nu * xs["sigma_f"]

# reactor_data now holds all the reorganized data


SyntaxError: ':' expected after dictionary key (1402424876.py, line 98)

# **Introduction to Fast Reactors**

# Zero Dimensional Model

## Motivation

$$
-\phi_g\nabla\cdot(D_g(\textbf{r})\nabla R(\textbf{r})) + \phi_g\Sigma_{t,g}(\textbf{r})R(\textbf{r}) = \frac{1}{k_\text{eff}}\chi_g(\textbf{r})\sum_{g'=0}^{G} \phi_{g'}R_{g'}(\textbf{r})(\nu\Sigma_f)_{g'}(\textbf{r})+\sum_{g'=0}^{G}\phi_{g'}R_{g'}(\textbf{r})\Sigma_{s, g'\to g}(\textbf{r})
$$

## Derivation

**Introduce a citation to the github repo**

The general N-dimensional multigroup neutron diffusion equation for a teady state reactor is defined by:

\begin{equation}
\frac{1}{v}\partial_t\phi(\textbf r)-\nabla\cdot(D_g(\textbf{r})\nabla\phi_g(\textbf{r})) + \Sigma_{t,g}(\textbf{r})\phi_g(\textbf{r}) = \frac{1}{k_\text{eff}}\chi_g(\textbf{r})\sum_{g'=0}^{G}\phi_{g'}(\textbf{r})(\nu\Sigma_f)_{g'}(\textbf{r})+\sum_{g'=0}^{G}\phi_{g'}(\textbf{r})\Sigma_{s, g'\to g}(\textbf{r}) 
\end{equation}

Under the ansatz that the group flux can be seperated in space and that $\phi_g$ is simply a scalar value, ie.,
$$\phi_g(\textbf{r}) = \phi_gR(\textbf{r})\quad \phi_g\in\mathbb{R}\quad R:=\mathbb{R}^N\mapsto\mathbb{R}$$

Substituting this into Eq. 1 gives,

\begin{equation}
-\phi_g\nabla\cdot(D_g(\textbf{r})\nabla R(\textbf{r})) + \phi_g\Sigma_{t,g}(\textbf{r})R(\textbf{r}) = \frac{1}{k_\text{eff}}\chi_g(\textbf{r})\sum_{g'=0}^{G} \phi_{g'}R_{g'}(\textbf{r})(\nu\Sigma_f)_{g'}(\textbf{r})+\sum_{g'=0}^{G}\phi_{g'}R_{g'}(\textbf{r})\Sigma_{s, g'\to g}(\textbf{r})
\end{equation}





If the reactor is homogenous, $R(\textbf r)$ is a fundamental eigenfunction of the Helmholtz equation. $$\nabla^2R(\textbf r)+B^2R(\textbf r)=0$$.
The spatial dependence on flux can be completely removed, and Eq. 2 can collapse into the zero-dimensional form,

$$D_gB^2\phi_g + \Sigma_{t,g}\phi_g = \frac{1}{k_\text{eff}}\chi_g\sum_{g'=0}^{G}\phi_{g'}(\nu\Sigma_f)_{g'}+\sum_{g'=0}^{G}\phi_{g'}\Sigma_{s, g'\to g}$$

# ----------------

Which in a simple-geometry, homogeneous reactor can be rewritten as:

$$D_gB^2\phi_g + \Sigma_{t,g}\phi_g = \frac{1}{k_\text{eff}}\chi_g\sum_{g'=0}^{G}\phi_{g'}(\nu\Sigma_f)_{g'}+\sum_{g'=0}^{G}\phi_{g'}\Sigma_{s, g'\to g}$$

Where $B^2$ is the geometric buckling of a reactor, which for a sphere is given by:

$$DB^2=\left(\frac{\pi}{\tilde{R}}\right)^2\quad\text{or}\quad B^2 = \left(\frac{\pi}{R}\right)^2\text{ when neglecting extrapolated distance}$$

And the diffusion coefficient, $D_g$, in an isotropic-scattering-system is given by:
$$D_g = \frac{1}{3\Sigma_{t,g}}$$

The diffusion equation can be written in as a linear system in the form:
$$\textbf{A}\Phi = \frac{1}{k}\textbf{B}\Phi$$

Where:

$$\textbf{A} = \begin{bmatrix}
D_1B^2+\Sigma_{t,1}-\Sigma_{s,1\to 1} & -\Sigma_{s,2\to 1} & \dots & -\Sigma_{s,G\to 1}\\
-\Sigma_{s,1\to 2} & D_2B^2+\Sigma_{t,2}-\Sigma_{s,2\to 2} & \dots & -\Sigma_{s,G\to 1}\\
\vdots & \vdots & \ddots & \vdots \\
-\Sigma_{s, 1\to G} & - \Sigma_{s, 2\to G} & \dots &  D_GB^2+\Sigma_{t,1}-\Sigma_{s,G\to G}
\end{bmatrix}$$

$$\textbf{B} = \begin{bmatrix}
\chi_1(\nu\Sigma_f)_1 & \chi_1(\nu\Sigma_f)_2 & \dots & \chi_1(\nu\Sigma_f)_G \\
\chi_2(\nu\Sigma_f)_1 & \chi_1(\nu\Sigma_f)_2 & \dots & \chi_2(\nu\Sigma_f)_G \\
\vdots & \vdots & \ddots & \vdots \\
\chi_G(\nu\Sigma_f)_1 & \chi_G(\nu\Sigma_f)_2 & \dots & \chi_G(\nu\Sigma_f)_G 
\end{bmatrix},\quad\Phi = \begin{bmatrix} \phi_1 \\ \phi_2 \\ \vdots \\ \phi_G\end{bmatrix}$$

The linear system can be rearranged to the form:
$$k\Phi=\textbf{A}^{-1}\textbf{B}\Phi$$

which is a solvable eigenpair problem where $\max(k) = k_\text{eff}$ and it's corresponding eigenvector, $|\Phi|$, is the flux distribution amungst the $G$ energy groups.


### Homogenization

In [ ]:
def homogenize_fuel_from_volume(pu_vol_frac = 0, fuel_element_outer_diameter = 6.25, clad_thickness = 0.04) :

    fuel_element_area = np.pi/4 * fuel_element_area**2
    fuel_area = fuel_element_area - np.pi * (fuel_element_outer_diameter/2 - clad_thickness)**2
    clad_area = fuel_element_area - fuel_area

    # Homogenize the fuel cross sections through volume
    fuel_xs = {}
    for xs_type in data["nuclides"]["pu239"]["xs"].keys():
        data["materials"]["fuel"]["xs"][xs_type] =  fuel_area * pu_vol_frac * data["nuclides"]["pu239"]["xs"][xs_type]     # add plutonium
        data["materials"]["fuel"]["xs"][xs_type] += fuel_area * (1-pu_vol_frac) * data["nuclides"]["u238"]["xs"][xs_type]  # add uranium
        data["materials"]["fuel"]["xs"][xs_type] += clad_area * (1-pu_vol_frac) * data["nuclides"]["fe"]["xs"][xs_type]    # add clad
        data["materials"]["fuel"]["xs"][xs_type] /= fuel_element_area                                                      # normalize

    


In [ ]:
print(data)

{'rho_fuel': 19.0, 'OD_fuel': 6.35, 'Th_clad': 0.04, 'delta_core': 20.0, 'p_th': 1200000000.0, 'phi_fuel_ratio': array([1.6 , 1.6 , 1.6 , 1.45, 1.45, 1.34, 1.34, 1.34]), 'phi_clad_ratio': array([1.01, 1.01, 1.01, 1.02, 1.02, 1.03, 1.03, 1.03]), 'phi_cool_ratio': array([0.9 , 0.9 , 0.91, 0.91, 0.91, 0.94, 0.94, 0.94]), 'lethargy_width': array([1.5, 1. , 1. , 1. , 1. , 1. , 3. , 0. ]), 'lower_energy_bounds': array([2.2e+06, 8.2e+05, 3.0e+05, 1.1e+05, 4.0e+04, 1.5e+04, 7.5e+02,
       0.0e+00]), 'sigma_tr_pb': array([1.5, 2.2, 3.6, 3.5, 4. , 3.9, 7.3, 3.2]), 'sigma_ng_pb': array([0.005 , 0.0002, 0.0004, 0.001 , 0.001 , 0.001 , 0.009 , 0.008 ]), 'sigma_f_pb': array([0, 0, 0, 0, 0, 0, 0, 0]), 'sigma_r_pb': array([0.623 , 0.6908, 0.4458, 0.29  , 0.35  , 0.3   , 0.04  , 0.    ]), 'nu_f_pb': array([0, 0, 0, 0, 0, 0, 0, 0]), 'sigma_tr_fe': array([ 2.2,  2.1,  2.4,  3.1,  4.5,  6.1,  6.9, 10.4]), 'sigma_ng_fe': array([0.02 , 0.003, 0.005, 0.006, 0.008, 0.012, 0.032, 0.02 ]), 'sigma_f_fe': array(